In [ ]:
# default_exp stats

# stats

> Módulo que contém os algoritmos para detecção de anomalias em dados univariados.

* Robust ZScore
* Tukey

In [ ]:
#hide
from nbdev.showdoc import *

## Pré-requisitos

In [ ]:
#export
import numpy as np

## MAD - Robust ZScore

Para detecção de anomalias o Zscore Robusto é mais adequado que o Zscore padrão, uma vez que a média pode ser fortemente influenciada por outliers.

In [ ]:
#export
class MAD():
    ''' 
    classe responsavel por implemetar zscore robusto
    para detecção de anomalias.
    '''
    def __init__(self):
        pass

    def _mad(self, x):
        ''' retorna o MAD(Median Absolute Deviation) para cada valor de **x** '''
        return (0.6745*(x - self.median))/self.mad
    
    def fit(self, x):
        ''' Calcula os parametros do Zscore Robusto(Median/MAD) para os valores de **x** '''
        self.mad = np.nanmedian(np.abs(x - np.nanmedian(x)))
        self.median = np.nanmedian(x)

    def predict(self, x, m=3.0):
        ''' retorna se os valores de **x** são outliers '''
        mad = self._mad(x)
        return x[np.abs(mad) > m]
    
    def fit_predict(self, x, m=3.0):
        ''' Calcula os parametros e retorno os valores
            de **x** que são outliers'''
        self.fit(x)
        return self.predict(x, m)

In [ ]:
show_doc(MAD.fit)

<h4 id="MAD.fit" class="doc_header"><code>MAD.fit</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>MAD.fit</code>(**`x`**)

Calcula os parametros do Zscore Robusto(Median/MAD) para os valores de **x** 

In [ ]:
show_doc(MAD.predict)

<h4 id="MAD.predict" class="doc_header"><code>MAD.predict</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>MAD.predict</code>(**`x`**, **`m`**=*`3.0`*)

retorna se os valores de **x** são outliers 

In [ ]:
show_doc(MAD.fit_predict)

<h4 id="MAD.fit_predict" class="doc_header"><code>MAD.fit_predict</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>MAD.fit_predict</code>(**`x`**, **`m`**=*`3.0`*)

Calcula os parametros e retorno os valores
de x que são outliers

## Tukey

In [ ]:
#export
class Tukey():
    ''' 
    classe responsavel por implemetar Tukey Method
    para detecção de anomalias.
    '''

    def __init__(self):
        self.iqr = None
        self.q1 = None
        self.q2 = None
        self.q3 = None
    
    def fit(self, x):
        ''' Calcula os parametros do Tukey(Q1,Q2,Q3) para os valores de **x** '''
        x = np.sort(x) 
        n = len(x)//2
    
        # calculando os quartiles
        self.q1 = np.nanmedian(x[:n])
        self.q2 = np.nanmedian(x)
        self.q3 = np.nanmedian(x[n:])
    
        self.iqr = self.q3 - self.q1
        self.min = self.q1 - 1.5*self.iqr
        self.max = self.q3 + 1.5*self.iqr
        
    def predict(self, x):
        ''' retorna se os valores de **x** são outliers '''
        return x[(x < self.min) | (x >= self.max)]
    
    def fit_predict(self, x):
        ''' Calcula os parametros e retorno os valores
            de **x** que são outliers'''
        self.fit(x)
        return self.predict(x)

In [ ]:
show_doc(Tukey.fit)

<h4 id="Tukey.fit" class="doc_header"><code>Tukey.fit</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>Tukey.fit</code>(**`x`**)

Calcula os parametros do Tukey(Q1,Q2,Q3) para os valores de **x** 

In [ ]:
show_doc(Tukey.predict)

<h4 id="Tukey.predict" class="doc_header"><code>Tukey.predict</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Tukey.predict</code>(**`x`**)

retorna se os valores de **x** são outliers 

#### TODO

* Arima
* DBSCAN
* Statistical Control Chart Techniques

## Funções de ajuda

In [ ]:
#export
def window(m, x):
    ''' calcula a probabilidade do intervalo ser um outlier
        com base nas medições individuais de cada medidor '''
    y_pred = m.predict(x)
    return y_pred.sum()/len(y_pred)

def window(m, x):
    ''' calcula a probabilidade do intervalo ser um outlier
        com base na soma das medições individuas '''
    y_pred = m.predict_proba(x)
    return y_pred.sum()